# Quick Start

## Common Settings


In [ ]:
from pydantic_settings import BaseSettings, SettingsConfigDict

class APISettings(BaseSettings):
    model_config = SettingsConfigDict(env_file='../.env', env_file_encoding='utf-8', extra='ignore')

    OPENAI_API_KEY: str
    ANTHROPIC_API_KEY: str
    MISTRAL_API_KEY:str
    
    LANGCHAIN_API_KEY: str
    LANGCHAIN_TRACING_V2: str = "true"

api_settings = APISettings()
for tuple in api_settings:
    print(f"{tuple[0]}:\t\t{tuple[1][:10]}")

# Using Language Models

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Spanish"),
    HumanMessage(content="How are you doing?"),
]

## OpenAI

In [ ]:
from langchain_openai import ChatOpenAI
chat_openai_model = ChatOpenAI(model="gpt-4",openai_api_key=api_settings.OPENAI_API_KEY)

chat_openai_model.invoke(messages).dict()

## Anthropic

In [ ]:
#%pip install langchain_anthropic
import getpass
import os

In [ ]:
from langchain_anthropic import ChatAnthropic
anthropic_model = ChatAnthropic(model="claude-3-sonnet-20240229",anthropic_api_key=api_settings.ANTHROPIC_API_KEY)

In [ ]:
anthropic_model.invoke(messages).dict()

In [ ]:
## Mistral AI

In [ ]:
%pip install -qU langchain-mistralai

In [ ]:
from langchain_mistralai import ChatMistralAI
mistral_model = ChatMistralAI(model="mistral-large-latest",mistral_api_key=api_settings.MISTRAL_API_KEY)
mistral_result=mistral_model.invoke(messages)
mistral_result.dict()

# Output Parsers

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
parser.invoke(mistral_result)

In [ ]:
chain = anthropic_model | parser
chain.invoke(messages)

# Tagging Tutorial

In [ ]:
import sys
!{sys.executable} -m pip install -r ../requirements.notebooks.txt

In [ ]:
%pip install --upgrade --quiet langchain langchain-openai
%pip install --upgrade --quiet python-dotenv
# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv
dotenv.load_dotenv()

In [ ]:
# Check if the API key is available
import os
assert 'OPENAI_API_KEY' in os.environ, 'Please set the OPENAI_API_KEY env var'
print(f'API key is set. Length: {len(os.environ["OPENAI_API_KEY"])}, starting with: {os.environ["OPENAI_API_KEY"][:8]}')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

In [ ]:
class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(
    Classification
)

tagging_chain = tagging_prompt | llm

In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
tagging_chain.invoke({"input": inp})

In [ ]:
sarcastic_sentences = [
    "Qué alegría, otra reunión que podría haber sido un correo electrónico.",
    "I'm so glad it's Monday. I was getting tired of all that weekend freedom.",
    "Wow, standing in line for hours is my absolute favorite pastime.",
    "Toll, noch eine Baustelle auf meinem Arbeitsweg. Ich liebe Staus am Morgen.",
    "Wunderbar, mein Nachbar übt wieder um Mitternacht Trompete. Wie melodisch.",
    ".יופי, אני אשב לי פה לבד בחושך",
    "!בדיוק מה שהייתי צריך היום"
]
for s in sarcastic_sentences:
    print(tagging_chain.invoke({"input": s}).dict(),f" {s[:20]}")

In [ ]:
class ClassificationEx(BaseModel):
    sentiment: str = Field(..., enum=["alegre", "excited", "neutral", "indifferent", "triste", "sarcástico", "enojado"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["Castellano", "English", "Deutsch", "עברית"]
    )

In [ ]:
llm_ext = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(
    ClassificationEx
)
tagging_chain_ex = tagging_prompt | llm_ext

inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
tagging_chain_ex.invoke({"input": inp})

for s in sarcastic_sentences:
    print(tagging_chain_ex.invoke({"input": s}).dict(),f" {s[:20]}")